In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
class Net_1(nn.Module):
    def __init__(self,feature_dim, hidden_dim, out_dim):
        super(Net_1,self).__init__()
        self.gru = nn.GRU(input_size = feature_dim, hidden_size = hidden_dim, num_layers = 2)
        self.conv = nn.Conv1d(hidden_dim, out_dim, kernel_size = 3)
    def forward(self,x):
        o1, hidden = self.gru(x)# o1 - time, batch, hidden_dim
        o2 = o1.permute(1,2,0)#o2 - batch, feature_dim, time
        output = self.conv(o2)
        o = output.max(dim = 2).values#时间维度上平均池化
        return o

In [3]:
model = Net_1(39,128,42)
x = torch.randn(135,5,39)#time, batch, feature_dim
y = model(x)
y.shape#(Batch,class_num) 

torch.Size([5, 42])

In [4]:
class Net_2(nn.Module):
    def __init__(self,feature_dim, hidden_dim, out_dim):
        super(Net_2,self).__init__()
        self.conv = nn.Conv1d(feature_dim, hidden_dim, kernel_size = 3)
        self.lstm = nn.LSTM(input_size = hidden_dim, hidden_size = out_dim, num_layers = 2)
    def forward(self,x):
        o1 = self.conv(x)# o1 - batch, hidden_dim, seq_len
        o2 = o1.permute(2,0,1)
        output, hidden = self.lstm(o2)#output是所有时间点最后一层隐藏层(time, batch, out_dim)
        o = output.mean(dim = 0)#时间维度上平均池化
        return o

In [5]:
model = Net_2(39,128,42)
x = torch.randn(5, 39, 135)#Batch, feature_dim, time
y = model(x)
y.shape#(Batch,class_num) 

torch.Size([5, 42])